# Mantenimiento proactivo: predicción de fallas con ML

Una empresa tiene una flota de dispositivos que transmiten lecturas diarias de sensores. Les gustaría crear una solución de mantenimiento predictivo para identificar de forma proactiva cuándo se debe realizar el mantenimiento. Este enfoque promete ahorros de costos con respecto al mantenimiento preventivo de rutina o basado en el tiempo, porque las tareas se realizan sólo cuando están justificadas.

El objetivo consiste en construir un modelo predictivo utilizando el aprendizaje automático para predecir la probabilidad de que falle un dispositivo. Al construir este modelo, asegúrese de minimizar los falsos positivos y los falsos negativos. La columna que está intentando predecir se llama falla con valor binario 0 para no falla y 1 para falla.


Fuente: https://www.kaggle.com/code/ahmettalhabektas/proactive-maintenance-predicting-failures-with-ml

# 1. Cragar la librerías necesarias

In [ ]:
##!pip install pandas-profiling -q

In [ ]:
!pip install ydata-profiling -q

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
## Pandas para manipular datos
import pandas as pd

## Matplotlib para graficar resultados
import matplotlib.pyplot as plt
%matplotlib inline
## Seaborn para visualización de datos
import seaborn as sns
## Numpy para las operaciones numéricas
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')


## Importando los algoritmos de ML supervisado
from sklearn.naive_bayes import GaussianNB, BernoulliNB  # For binary classification
##from sklearn.naive_bayes import MultinomialNB  # For multi-class classification
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors classifier
## Support Vector Machines
from sklearn.svm import SVC
## Árbol de decisión
from sklearn.tree import DecisionTreeClassifier
## Regresión logística
from sklearn.linear_model import LogisticRegression
##from sklearn.ensemble import GradientBoostingClassifier  # Gradient Boosting classifier

## Métricas para evaluación de modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

## Herramienta para seprar datos de entrenamiento y testeo (validación)
from sklearn.model_selection import train_test_split

# 2. Lectura de datos

In [ ]:
## Definir acceso al Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Define la ruta de acceso y archivo con datos
path = '/content/drive/MyDrive/AA - Salfa/Casos/'
filename = 'DatosFallas-V1.xlsx'
df = pd.read_excel(path+filename)

# Análisis Exploratorio de Datos (EDA)

Entender el comportamiento de cada variable individualmente calculando recuentos de frecuencia, visualizando las distribuciones, etc. También las relaciones entre las diversas combinaciones de las variables predictoras y de respuesta creando diagramas de dispersión, correlaciones, etc.

- Comprender qué variables podrían ser importantes para predecir la Y (respuesta);
- Generar insights que nos permitan una mayor comprensión del contexto y desempeño del negocio.

EDA suele ser parte de cada proyecto de aprendizaje automático/modelado predictivo, especialmente con conjuntos de datos tabulares.

In [ ]:
df.profile_report()

In [ ]:
## Entrega la cantida de filas y columnas
df.shape

In [ ]:
## Borra las filas duplicadas
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
## Grafica dos métricas
plt.figure(figsize=(10, 6))
plt.scatter(df['Metrica 7'], df['Metrica 8'], alpha=0.5)
plt.title('Gráfico entre Metrica 7 y Metrica 8')
plt.xlabel('Metrica 7')
plt.ylabel('Metrica 8')
plt.grid(True)
plt.show()

Al parecer hay muchos datos concentrados al inición de los valores, lo cual genera distribuciones cargadas hacia los valores iniciales.

Entonces, se rectifican los valores realizando una transformación logoritmica de ellos

In [ ]:
for num in ["2","3","4","7","8","9"]:
    df[f'Metrica {num}'] = np.log1p(df[f'Metrica {num}'])

In [ ]:
## Gráfico de dos métricas
plt.figure(figsize=(10, 6))
plt.scatter(df['Metrica 7'], df['Metrica 8'], alpha=0.5)
plt.title('Gráfico entre Metrica 7 y Metrica 8')
plt.xlabel('Metrica 7')
plt.ylabel('Metrica 8')
plt.grid(True)
plt.show()

Hay una fuerte correlación entre la métrica 7 y 8, por lo cual no agrega ninguna información manejar las dos. Se procede a eliminar la metrica 8

In [ ]:
df.drop("Metrica 8", axis = 1, inplace = True)

Se resumen los datos disponibles

In [ ]:
def summarize_data(df):
    print("Número de filas y columnas:", df.shape)
    print("\nColumnas de datos:", df.columns)
    print("\nTipo de datos y valores perdidos:")
    print(df.info())
    print("\nResumen de las columnas numéricas:")
    print(df.describe())
    print("\nValores perdidos:")
    print(df.isnull().sum())
    print("\nValores únicos en la columna 'failure':")
    print(df['Falla'].value_counts())

## Llama la función de resumen
summarize_data(df)

Revisamos cuántos dispositivos estamos monitoreando, primero que nos entregue la columna de los nombres de los dispositivos

In [ ]:
df["Dispositivo"]

In [ ]:
df

Ahora los dispositivos diferentes existentes en el monitoreo

**Nueva Características**

Creamos la características llamada 'device_model' que contiene los diferentes dispositivos (4 primeros caractéres) existentes en el monitoreo, y nos muestra la cantidad de registros asociados

In [ ]:
df["Modelo_Dispositivo"] = df["Dispositivo"].apply(lambda x : x[:4])
df["Modelo_Dispositivo"].value_counts()

Creamos otra variable con los cuatro segundos caractéres de los nombre de los dispositivos

In [ ]:
df["Resto_Dispositivo"] = df["Dispositivo"].apply(lambda x : x[4:])
df["Resto_Dispositivo"].value_counts()[:20]

Ahora eliminamos la columna 'device' ya que fue separada en dos diferentes ('device_model'y 'device_rest')

In [ ]:
df.drop("Dispositivo",axis=1,inplace=True)

In [ ]:
df

**Histogramas de fallas y no fallas**

Construimos un histograma de la cantidad de fallas y no fallas por dispositivo

In [ ]:
## Creamos dos gráficos
plt.figure(figsize=(12, 6))
## Graficamos la distribución (histograma) de las fallas por dispositivos (failure=1)
plt.subplot(1, 2, 1)
sns.countplot(x='Modelo_Dispositivo', data=df.loc[df['Falla'] == 1])
plt.title('Distribución de falla (Falla=1) por dispositivo')

## Graficamos la distribución de la no falla (failure=0) por dispositivo
plt.subplot(1, 2, 2)
sns.countplot(x="Modelo_Dispositivo", data=df.loc[df["Falla"] == 0])
plt.title('Distribución de no falla (Falla=0) por dispositivo')

## Ajustamos el layout
plt.tight_layout()

## Mostramos los gráficos
plt.show()

Al parecer el dispositivo Z1F2 presentan sólo casos de no falla, lo cual no nos permite generar la clasificación en forma adecuada (no tiene datos de fallas).

Entonces, decidimos eliminar las filas asociadas a dicho dispositivo

In [ ]:
df.drop(df.loc[df["Modelo_Dispositivo"]=="Z1F2"].index,axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
df.tail()

Note que de los 124.494 datos originales (filas), debido a la eliminacón de duplicados y ahora de dispositivos sin fallas sólo quedan 124.242 filas

**Entendiendo device_rest**

Ahora queremos saber la distribución asociada a la característica devide_rest, entonces graficamos para el caso de fallas solamente (Falla=1)

In [ ]:
## Crea el gráfico para este análisis
plt.figure(figsize=(12, 6))
sns.countplot(x="Resto_Dispositivo", data=df.loc[df["Falla"] == 1])
plt.title('Distribución de la no falla (Falla=1) con respecto al dispositivo')

plt.tight_layout()

plt.show()

La verdad no entrega mayor información, ya que no presenta una cantidad de más de un dato en cuento a las fallas. Por esta razón, decidimos eliminar esta variable

In [ ]:
df.drop("Resto_Dispositivo", axis = 1, inplace = True)
df.sample(5)

## Histograma de las diferentes características

In [ ]:
## Crea el histograma de las diferentes métricas
## considera el caso de las no falla (failure = 0)
plt.figure(figsize=(4*5, 2*5))
print("Distribución para no falla (Falla = 0)")
mask = df.Falla == 0
for i, col in enumerate(['Metrica 1', 'Metrica 2', 'Metrica 3', 'Metrica 4', 'Metrica 5', 'Metrica 6', 'Metrica 7', 'Metrica 9']):
    plt.subplot(2, 4, i + 1)
    sns.histplot(data=df.loc[mask], x=col, kde=True)
    plt.title(f'Distribución de {col}')
plt.tight_layout()

In [ ]:
## Ahora las distribuciones para falla (failure=1)
plt.figure(figsize=(20, 10))
print("Distribución para fallas (failure = 1)")
mask = df.Falla > 0
for i, col in enumerate(['Metrica 1', 'Metrica 2', 'Metrica 3', 'Metrica 4', 'Metrica 5', 'Metrica 6', 'Metrica 7', 'Metrica 9']):
    plt.subplot(2, 4, i + 1)
    sns.histplot(data=df.loc[mask], x=col, kde=True)
    plt.title(f'Distribución de {col}')
plt.tight_layout()

**Manejo de fechas**

Se transforma el formato de las fechas a uno que entregue formato manejable

In [ ]:
## Convierte 'date' a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

## Extrae y formatea el mes para graficar
df['mes'] = df['Fecha'].dt.to_period('M')
df['mes'] = df['mes'].dt.strftime('%Y-%m')

## Crea un gráfico de línea para visualizar 'failure' sobre los meses
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='mes', y='Falla')
plt.xticks(rotation=45)
plt.title("Falla en el tiempo (meses)")

In [ ]:
## Extrae y formatea la columna 'week'para graficar
df['semana'] = df['Fecha'].dt.to_period('W')
df['semana'] = df['semana'].dt.strftime('%Y-%U')

## Grafica las fallas por semana
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='semana', y='Falla')
plt.xticks(rotation=45)
plt.title("Fallas por semana")

Mostramos los nuevos datos que incluyen una columna o característica 'mes' y 'semana'

In [ ]:
df

## Correlaciones entre datos

Sólo para aquellas columnas o características numéricas se calcula las correlaciones cruzadas existentes

In [ ]:
## Seleccionamos las columnas numéricas para la matriz de correlaciones
numeric_cols = df.select_dtypes(include=[np.number])

## Calcula la matriz de correlación
correlation_matrix = numeric_cols.corr()

## Crea un mapa de calor para visualizar la matriz de correlación
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Matriz de Correlación")

Dado que la falla es generalmente cero (no falla), buscar la correlación no sería eficiente

In [ ]:
## Calculamos un gráfico de frecuencia de falla y no falla
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Falla')
plt.title("Distribución de falla")

Como es posible observar desde el gráfico la cantidad de no falla son muy superior a la cantidad de fallas en los datos. Esta situación de ser utilizada directamente traerí un problema de sub muestreo para el caso de las fallas.

Es razonable seguir con los datos tal cual como están?, para que situación sería razonable?


## Estudio de fallas

Queremos revisar si las fallas se dan en días específicos de las semana, fin de semana o días del mes. Para ello, creamos columnas o características adicionales que incluyan el día de la semana, el día del mes y si es fin de semana

In [ ]:
## Obtenemos el día de la semana, día del mes y si es fin de semana desde
## la columna fecha
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['dia_semana'] = df['Fecha'].dt.dayofweek
df['dia_mes'] = df['Fecha'].dt.day
df['fin_semana'] = df['dia_semana'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
## Ahora creamos gráficos para visualizar las distribuciones de falla
## por día de la semana, día del mes y fin de semana
plt.figure(figsize=(15, 5))

## Distribución por día de la semana
plt.subplot(131)
sns.countplot(data=df, x='dia_semana', palette='Set3')
plt.title("Distribución por día de semana")
plt.xlabel("Día de semana")
plt.ylabel("Cantidad")

## Distribución por día del mes
plt.subplot(132)
sns.countplot(data=df, x='dia_mes', palette='Set3')
plt.title("Distribución por día del mes")
plt.xlabel("Día del mes")
plt.ylabel("Cantidad")

## Distribución por fin de semana
plt.subplot(133)
sns.countplot(data=df, x='fin_semana', palette='Set3')
plt.title("Distribución por fin de semana")
plt.xlabel("Fin de semana (1) or Día de semana (0)")
plt.ylabel("Cantidad")

## Muestra los gráficos
plt.tight_layout()
plt.show()

In [ ]:
## Extraemos el número del mes y número de la semana y eliminamos 'date'
df['mes'] = df['Fecha'].dt.month
df['semana'] = df['Fecha'].dt.isocalendar().week
df = df.drop(['Fecha'], axis=1)

## Chequeamos los datos del nuevo dataframe
df.info()

In [ ]:
## Para aplicar la codificación One Hot Encoding se utiliza la función pd.get_dummies()
## Cada variable se convierte en tantas variables 0/1 como valores diferentes haya.
## Las columnas de la salida reciben el nombre de un valor; si la entrada es un DataFrame,
## el nombre de la variable original se antepone al valor
## La opción drop_first indica que se deben sacar los dummies k-1 de los k niveles
## categóricos eliminando el primer nivel
df = pd.get_dummies(df,drop_first=True)
df

## Extración de valores aleatorios de datos

Debido al fuerte desbalance entre no fallas (124.388) y fallas (106) originalmente que se observó anteriormente, entonces muestreamos desde las filas de no falla un subconjunto que nos permita trabajar sin sesgo.

In [ ]:
## Importanmos una librería de submuestreo aleatorio
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
## Crea una copia del dataframe 'df'
X = df.copy()

## Crea una copia de la variable 'Falla" en 'Y'
Y = df["Falla"]

## Elimina desde la copia 'X' la columna 'Falla'
X.drop("Falla", axis=1, inplace=True)

In [ ]:
## Creamos una instancia de RandomUnderSampler con un valor para random state
rus = RandomUnderSampler(random_state=42)

## Realizamos el muestreo under-sampling y obtenemos las filas resampled de
## características 'X' y variable objetivo 'Y'
X_resampled, y_resampled = rus.fit_resample(X, Y)

In [ ]:
## Creamos un nuevo dataframe 'under_sample' copiando las muestras resampled
## de las características y agregamos la columna 'failure'
under_sample = X_resampled.copy()
under_sample["failure"] = y_resampled

In [ ]:
## Mostramos el nuevo data frame desde los datos under-sampled
under_sample.sample(10)

In [ ]:
under_sample.shape

Notamos que la cantidad de datos disponibles para el trabajo de modelos de ML supervisados es de sólo 212 filas, pero esto debería dar una muestra equilibrada de los datos

In [ ]:
## Finalmente, graficamos las no fallas y fallas en un gráfico de frecuencia
## desde el nuevo dataframe under-sampled
plt.figure(figsize=(6, 4))
sns.countplot(data=under_sample, x='failure')
plt.title("Distribución de las fallas")

Observamos ahora una muestra equitativa entre las no fallas y fallas

# Modelo de ML

Dado que tenemos los datos que nos permiten construir modelos de ML, entonces procedemos a construirlos.

Desde los datos (under_sample) lo que debemos hacer es separar los datos en aquellos que se utilizarán para el entrenamiento de los diversos modelos (x_train e y_train) que representarán un 20% de todos los datos disponibles (212 filas), y aquellos de validación o testeo (x_test e y_test). Para tales efectos se utiliza la librería train_test_split.

En forma adicional, y de manera de eliminar los efectos de escalas diferentes de los datos, se utiliza un"escalador" que tiene por objetivo escalar los datos de manera de hacerlos comparables en escala. Para tales efectos se utiliza la libería StandarScaler

In [ ]:
## Importamos las librería necesarias para crear los datos de los modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Separamos los datos en 'caracteríasticas'y 'variable a explicar'
X_norm = under_sample.drop(['failure'], axis=1)
y_norm = under_sample['failure']

## Separamos los datos en entrenamiento (train) y validación (test)
x_train, x_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

## Ahora, realizamos el escalamiento de los datos de entrenamiento y test
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

**Importamos la librería para los modelos**

In [ ]:
## Importamos las librerías de los modelo de clasificación de ML
from sklearn.naive_bayes import GaussianNB, BernoulliNB
#######from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression #####, SGDClassifier
from sklearn.ensemble import RandomForestClassifier #####, GradientBoostingClassifier, , AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

**Creamos los modelos**

In [ ]:
## Define una función con diferentes clasificadores y retorna
## un dataframe con las métricas de evaluación
def evaluate_model(x_train, y_train, x_test, y_test):
    ## Lista de clasificadores a utilizar
    classifiers = [
        RandomForestClassifier(),
        DecisionTreeClassifier(),
        KNeighborsClassifier(),
        GaussianNB(),
        BernoulliNB(),
        SVC(),
        LogisticRegression(),
    ]

    ## Define los nombre de los clasificadores
    classifier_names = [
        'RandomForest',
        'DecisionTree',
        'KNeighbors',
        'GaussianNB',
        'BernoulliNB',
        'SVC',
        'LogisticRegression',
    ]
    ## Crea una dataframe vacío para las métricas
    metrics = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1'], index=classifier_names)

    ## Evalúa cada clasificador y almacena las métricas
    for i, clf in enumerate(classifiers):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        metrics.loc[classifier_names[i], 'Accuracy'] = accuracy
        metrics.loc[classifier_names[i], 'Precision'] = precision
        metrics.loc[classifier_names[i], 'Recall'] = recall
        metrics.loc[classifier_names[i], 'F1'] = f1

    ## Ordena las métricas por exactitud (accuracy) en orden descendente
    metrics = metrics.sort_values(by='Accuracy', ascending=False)

    return metrics

In [ ]:
## Paso 4: Llama a la función para evaluar los modelos
metrics = evaluate_model(x_train, y_train, x_test, y_test)

In [ ]:
metrics

# Instalamos optuna

Optuna es un marco de software de optimización automática de hiperparámetros, especialmente diseñado para el aprendizaje automático

Referencia: https://optuna.readthedocs.io/en/stable/

In [ ]:
!pip install optuna -q

In [ ]:
import optuna

In [ ]:
## Crea un objeto de Optuna Study
def create_study(objective):

    ## Establezca el nivel de registro en ADVERTENCIA para suprimir la salida innecesaria
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize')  # We want to maximize accuracy

    ## Corra la optimización
    study.optimize(objective, n_trials=100)  ## Es posible ajustar en número de intentos

    # Obtenga los mejores hiperparámetros
    best_params = study.best_params
    best_f1 = study.best_value
    print(f'Best hyperparameters: {best_params}')
    print(f'Best f1 score: {best_f1}')
    return best_params

# Random Forest

In [ ]:
## Define la función objetivo a optimizar
def objective_rf(trial):
    ## Definir los hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 10, 150)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_uniform('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    max_features = trial.suggest_categorical('max_features', ['log2', 'sqrt'])

    ## Crear y entrenar el RandomForestClassifier con los hiperparámetros sugeridos
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    ## Entrene el clasificardor con datos de entrenamiento
    clf.fit(x_train, y_train)

    ## Haga la predicción con datos de testeo
    y_pred = clf.predict(x_test)

    ## Calcule el F1-Score como objetivo a maximizar
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
## Cree el objeto para Random Forest
best_params = create_study(objective_rf)
## Ajuste con los hiperparámetros
best_rf = RandomForestClassifier(**best_params, random_state=42)
## Obtenga el mejor
y_pred_rf = best_rf.fit(x_train, y_train).predict(x_test)

# Árbol de desción

In [ ]:
## Define la función objetivo a optimizar
def objective_dt(trial):
    ## Define los hiperparámetros y rangos
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 4, 32, log=True)
    min_samples_split = trial.suggest_uniform('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)

    ## Crea el clasificardor DecisionTree con los hiperparámetros sugeridos
    clf = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42  # Set a random state for reproducibility
    )

    ## Ajusta el clasificador con datos de entrenamiento
    clf.fit(x_train, y_train)

    ## Predice con datos de testeo
    y_pred = clf.predict(x_test)

    ## Calcula el F1-Score con función objetivo
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
best_params = create_study(objective_dt)
best_dt = DecisionTreeClassifier(**best_params, random_state=42)
y_pred_dt = best_dt.fit(x_train, y_train).predict(x_test)

In [ ]:
## Dibijamos el árbol seleccionado
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

## Grafica el árbol
plt.figure(figsize=(20, 10))
plot_tree(best_dt, feature_names=df.drop("Falla",axis=1).columns.to_list(), class_names=["Non-Failure", "Failure"], filled=True, fontsize=10)
plt.show()

# Vecino más Cercano (K Neighbors Classifier)

In [ ]:
def objective_knn(trial):
    ## Define los hiperparámetros
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 20),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'p': trial.suggest_int('p', 1, 2),  ## p=1 para la distancia tipo Manhattan distance,
                                            ## p=2 para la distancia Euclidiana
    }

    ## Inicializa el clasificador con los hiperparámetros
    clf = KNeighborsClassifier(**params)

    ## Entrena el clasificador con los datos de entrenamiento
    clf.fit(x_train, y_train)

    ## Predice con los datos de test
    y_pred = clf.predict(x_test)

    ## Calcula el F1-Score como función objetivo a maximizar
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
best_params=create_study(objective_knn)
best_knn = KNeighborsClassifier(**best_params)
y_pred_knn = best_knn.fit(x_train, y_train).predict(x_test)

# Naive Bayes

In [ ]:
best_gnb = GaussianNB()
y_pred_gnb = best_gnb.fit(x_train, y_train).predict(x_test)

# BernoulliNB

In [ ]:
def objective_bnb(trial):
    # Define hyperparameters to optimize
    params = {
        'alpha': trial.suggest_loguniform('alpha', 1e-10, 1.0),
        'binarize': trial.suggest_float('binarize', 0.0, 1.0),
        'fit_prior': trial.suggest_categorical('fit_prior', [True, False]),
    }

    # Initialize the classifier with hyperparameters
    clf = BernoulliNB(**params)

    # Train the classifier on the training data
    clf.fit(x_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(x_test)


    # Calculate F1 score as the objective to maximize
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
best_params=create_study(objective_bnb)
best_bnb = BernoulliNB(**best_params)
y_pred_bnb=best_bnb.fit(x_train, y_train).predict(x_test)

# Support Vector Machine (SVC)

In [ ]:
def objective_svc(trial):
    ## Define los hiperparámetros para optimizar
    params = {
        'C': trial.suggest_loguniform('C', 1e-3, 1e3),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
        'degree': trial.suggest_int('degree', 2, 5) if trial.params['kernel'] == 'poly' else 1,
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']) if trial.params['kernel'] in ['rbf', 'poly', 'sigmoid'] else 'scale',
    }

    ## Inicializa el clasificar con los hiperparámetros
    clf = SVC(**params, random_state=42)

    ## Entrena el clasificador con los datos de entrenamiento
    clf.fit(x_train, y_train)

    ## hace predicciones con datos de testeo
    y_pred = clf.predict(x_test)

    ## Calcula el F1-Score como el objetivo a optimizar
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
best_params=create_study(objective_svc)
best_svc = SVC(**best_params)
y_pred_svc = best_svc.fit(x_train, y_train).predict(x_test)

# Regresión Logística

In [ ]:
def objective_lr(trial):
    ## Define los hiperparámetros para optimizar
    params = {
        'C': trial.suggest_loguniform('C', 1e-5, 1e5),
        'solver': trial.suggest_categorical('solver', ['liblinear', 'lbfgs']),
    }

    ## Inicializa el clasificador con los hiperparámetros
    clf = LogisticRegression(**params, random_state=42)

    ## Entrena el classificador con datos de entrenamiento
    clf.fit(x_train, y_train)

    ## predice con los datos de testeo
    y_pred = clf.predict(x_test)

    ## Calcula el F1-Score como la función a optimizar
    f1 = f1_score(y_test, y_pred)

    return f1

In [ ]:
best_params=create_study(objective_lr)
best_lr = LogisticRegression(**best_params)
y_pred_lr = best_lr.fit(x_train, y_train).predict(x_test)

# Comparación entre modelos

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Función para calcular las métricas
def calculate_evaluation_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    return precision, recall, f1, accuracy

## Función que muestra la matriz de confusión
def plot_confusion_matrix(ax, y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", linewidths=0.5, linecolor="black", cbar=False, xticklabels=["Non-Failure", "Failure"], yticklabels=["Non-Failure", "Failure"], ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title(title)

In [ ]:
## Inicializa las variables para la mejor métrica
best_model = ""
best_f1 = 0.0
best_precision = 0.0
best_recall = 0.0
best_accuracy = 0.0

## Crea subgráficos de 2x2 (matriz de confusión)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(3*5, 4*5))

## Grafica para cada modelo la matriz de confusión
models = [
    ("Model Random Forest", y_pred_rf),
    ("Decison Tree", y_pred_dt),
    ("KNN", y_pred_knn),
    ("GaussianNB", y_pred_gnb),
    ("BernoulliNB",y_pred_bnb),
    ("SVC", y_pred_svc),
    ("LogisticRegression", y_pred_lr),
]

for (model_name, y_pred), ax in zip(models, axes.flatten()):
    plot_confusion_matrix(ax, y_test, y_pred, f"Confusion Matrix - {model_name}")

    ## Calcula la evaluación de métricas
    precision, recall, f1, accuracy = calculate_evaluation_metrics(y_test, y_pred)

    ## Imprime las métricas para cada modelo
    print(f"\nModel: {model_name}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    ## Actualiza el mejor modelo basado en el F1-Score
    if f1 > best_f1:
        best_f1 = f1
        best_model = model_name
        best_precision = precision
        best_recall = recall
        best_accuracy = accuracy

## Muestra los gráficos
plt.tight_layout()
plt.show()

In [ ]:
## Imprime el mejor modelo de acuerdo al F1-Score
print("\n *** Mejor Modelo encontrado ***\n")
print(f"Modelo: {best_model}")
print(f"Precision: {best_precision:.4f}")
print(f"Recall: {best_recall:.4f}")
print(f"F1 Score: {best_f1:.4f}")
print(f"Accuracy: {best_accuracy:.4f}")

# Curva ROC

Una curva característica operativa del receptor , o curva ROC , es un gráfico que ilustra el rendimiento de un modelo de clasificador binario (también puede usarse para clasificación de múltiples clases) en valores de umbral variables.

La curva ROC es el gráfico de la tasa de verdaderos positivos (TPR) frente a la tasa de falsos positivos (FPR) en cada ajuste de umbral.

In [ ]:
from sklearn.metrics import roc_curve, auc
## Se utiliza el modelo Gaussian Naive (y_pred_gnb)
## Podrían utilizarse otros modelos tales como
## - Árbol de decisión: y_pred_dt
## - Random Forest: y_pred_rf
## - SVC: y_pred_svc
##
fpr, tpr, thresholds = roc_curve(y_test, y_pred_gnb)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc = 'lower right')
plt.show()

# Cómo realizamos predicciones?

En esta sección tomaremos un modelo ya optimizado con desde el punto de vista de los hioperparámetros y haremos predicciones para que nos indique si habrá o no falla.

Para ello, seleccionamos. un modelo, por ejemplo tomamos el de Naive Bayes y le presentamos un posible escenario, es decir, un conjunto de métricas para que nos diga si hay falla (1) o no hay falla (0).

Hay que notar que los valores que le entregemos deben estar escalados para que los entienda como válidos


Datos a ingresar

- Metrica 1 -> 241295360
- Metrica 2 -> 0.000000
- Metrica 3 -> 0.000000
- Metrica 4 -> 0.000000
- Metrica 5 -> 6
- Matrica 6 -> 305202
- Metrica 7 -> 0.000000
- Metrica 9 -> 0.000000
- mes -> 2
- semana -> 8
- dia_semana -> 2
- dia_mes -> 22  
- fin_semana -> 0
- Modelo_Dispositivo_S1F1 -> False
- Modelo_Dispositivo_W1F0 -> True
- Modelo_Dispositivo_W1F1 -> False
- Modelo_Dispositivo_Z1F0 -> False
- Modelo_Dispositivo_Z1F1 -> False

In [ ]:
## Cosntruimos el objeto de dato de entrada
x_check = np.array([241295360, 0.0, 0.0, 0.0, 6, 305202, 0.0, 0.0, 2, 8, 2, 22, 0, False, True, False, False, False]).reshape(1, -1)

## Escalamos el datos para tenerlo en lo que revisó el modelo
scaler = StandardScaler()
x_valores = scaler.fit_transform(x_check)
x_test = scaler.transform(x_test)

## Realizamos la predicción
y_pred_gnb = best_gnb.predict(x_valores)
## Imprimimos la predicción
print('Posibilidad de falla', y_pred_gnb[0])

# Guardando y recuperando el modelo

In [ ]:
import joblib
joblib.dump(best_gnb, path+'modeloNB')


In [ ]:
model2 = joblib.load(path+'modeloNB')

val = model2.predict(x_valores)
print('Posibilidad de falla', val[0])